## スケーリング(拡大・縮小)

In [ ]:
import cv2
#画像を読み込む。第二引数は 0:グレースケール、1:カラー画像 -1:カラー＋アルファ
img = cv2.imread('Images/arfukuoka.png',1)

#方法1 :fx,fyを使って縦横の倍率を決める
res=cv2.resize(img,None,fx=1.3, fy=1.3)
#方法2 :縦横のサイズを直打ちして指定(補間方法はオプション)
res2=cv2.resize(img,(300,200),interpolation = cv2.INTER_CUBIC)

#画像をWindowで表示
cv2.imshow('image',img)
cv2.imshow('method1',res)
cv2.imshow('method2',res2)

#キー入力を待つ
cv2.waitKey(0)
#何かしらのキーを打つとウィンドウを閉じる
cv2.destroyAllWindows()
#全ての処理を終了する前にちょっと待つ。
cv2.waitKey(1)

### 補足
結果は以下の通り。縦横のサイズは倍数でもピクセル数でも指定可能。<br>
<img src="./Results/scaling.png" width="560" >
<br>
interpolationについては画像を拡大・縮小に伴って増加・減少した画素の各色を補う(補間する)方法を指定するために使用。<br>
直線的な補間を用いる場合は<b>cv2.INTER_LINEAR</b>を、滑らかな補間を用いる場合は<b>cv2.INTER_CUBIC</b>などを選べる。<br>
他にも補間方法は選べるので公式で詳細をご覧ください。<br>
http://opencv.jp/opencv-2.1/cpp/geometric_image_transformations.html#cv-resize
<br><br>
ちなみにinterpolationは指定しなくてもOK。その場合はデフォルトのINTER_LINEARとなる。

## 並進(x,y方向の移動)

In [ ]:
import cv2
import numpy as np
#画像を読み込む。第二引数は 0:グレースケール、1:カラー画像 -1:カラー＋アルファ
img = cv2.imread('Images/arfukuoka.png',1)
height,width,_=img.shape

#並進行列(tx=100, ty=70)
M = np.float32([ [1,0,100] , [0,1,70] ])

#warpAffineでimgをMの情報に基づいて変換(移動)する。
dst = cv2.warpAffine(img,M,(width,height))

#画像をWindowで表示
cv2.imshow('image',img)
cv2.imshow('result',dst)

#キー入力を待つ
cv2.waitKey(0)
#何かしらのキーを打つとウィンドウを閉じる
cv2.destroyAllWindows()
#全ての処理を終了する前にちょっと待つ。
cv2.waitKey(1)

### 補足
実行した結果、以下の通りx方向に100pixel、y方向に70pixel移動する。<br>
<img src="./Results/translation.png" width="400" >
<br>
移動のパラメータは行列という形でMに格納され、100や70と書かれた箇所を変更すると移動量も変わる。<br>
実際の変換(移動)はwarpAffineという関数で実行され、Mで設定した情報に基づいて第1引数の画像に変換をかける。<br>
本サンプルの設定だと移動後にウィンドウの外側の情報が破棄されるが、warpAffine関数の第3引数の(width,height)の値を変えて出力画像のサイズを大きくすれば全容を見ることも可能。<br>

### Mの中身の意味
元画像の各ピクセルの座標を(x,y)、移動量を(tx,ty)、移動後の点を(x',y')とすると移動は以下のように表せる。
<img src="./Results/formula1.png" width="100" ><br>
この式は変数(x,y)、定数(tx,ty)が混ざっているので下図のように入力・処理・出力に分けて式を整理したいとする。<br>
要するにプログラミングで言うところの引数・関数(メソッド)・戻り値のように役割を分ける。<br>
<img src="./Results/trans_overview.png" width="300" ><br>
これを数学の表現方法と照らし合わせて表現すると厳密さに目をつぶれば以下の通りになる。<br>
<img src="./Results/trans_formula.png" width="290" ><br>
上記の表現方法に合わせるため、まず元になる式を下記のように書き換える。<br>
ここで入力する変数(最後の1は便宜上記述)を青字、処理内容を赤字で示して式を見ると、あらゆるxyに対して決められた値(赤字)をかけたり足したりして結果を返していることが分かる。<br>
<img src="./Results/formula2.png" width="170" ><br>
これを行列で表すと以下のように処理用のパラメータと入力の変数を分離できる。なおパラメータの並びは上記の式の赤字部分の順番と対応している。<br>
<img src="./Results/formula3.png" width="220" ><br>
Pythonでこの行列を扱う場合は上式の一行目と二行目をそれぞれ配列化して二重配列としてまとめた以下の表記となる。<br>
<img src="./Results/transMatArray.png" width="300" ><br>
最後にこの行列MをwarpAffineに渡すことにより、「各ピクセルに対して行列Mのルールに従った処理をしてね」ということを設定できる。<br><br>
ここまで並進行列の意味を解説をしてきましたが、OpenCVを触る上でこの式を自力で導出できる必要はなく、tx,tyに相当するところを変更すれば移動ができるということを知っていれば十分です。あくまでプログラムを組む際に他の1や0のようなパラメータを同時に書かなくてはいけない理由を理解する上での基礎知識にしてもらえればOKです。

[余談]<br>
1行1列(左上)のパラメータはxに必ずかけられる数です。これを2にすると入力されたx座標が2倍になって返ってきます。つまりresizeで拡大したのと同じ効果が得られ、さらにtx,tyによる移動も行われます。つまり、この行列を使いこなすと移動と拡大縮小、次のサンプルで扱う回転などを1つの行列で一気に扱うこともできるようになります。

## 回転

In [ ]:
import cv2
import numpy as np
#画像を読み込む。第二引数は 0:グレースケール、1:カラー画像 -1:カラー＋アルファ
img = cv2.imread('Images/arfukuoka.png',1)
height,width,_=img.shape

#回転行列(画像の真ん中を軸に60°回転)
M = cv2.getRotationMatrix2D((width//2,height//2),60,1)

#warpAffineでimgをMの情報に基づいて変換(回転)する。
dst = cv2.warpAffine(img,M,(width,height))

#画像をWindowで表示
cv2.imshow('image',img)
cv2.imshow('result',dst)

#キー入力を待つ
cv2.waitKey(0)
#何かしらのキーを打つとウィンドウを閉じる
cv2.destroyAllWindows()
#全ての処理を終了する前にちょっと待つ。
cv2.waitKey(1)

### 補足
実行した結果、以下の通り画像の中央座標を中心に反時計回りに60°回転する。<br>
<img src="./Results/rotation.png" width="400" >
<br>
基本的な手順は並進と同じだが、変換行列のMの作り方が若干異なる。<br>
まず前提として、回転を伴う操作も並進と同様に行列で表現することができ以下のように記述される。(純粋な回転の場合はtx=ty=0)<br>
そのためsinやcosの値を計算して配列の各要素に代入すれば回転行列を表す配列Mを作ることは可能である。<br>
<img src="./Results/rotationMat.png" width="280" >
<br>
しかし回転するだけなのに上記の行列を覚えたりsin,cosを計算したりは面倒。回転する角度を指定するだけで変換してくれた方が多くの場合は楽です。<br>
そこでOpenCVでは「どこを中心」に「何度回転するか」を指定すれば変換行列を生成してくれるgetRotationMatrix2D関数を用意しており、これを利用すればOK。<br>

## アフィン変換(変形)

In [ ]:
import cv2
import numpy as np
#画像を読み込む。第二引数は 0:グレースケール、1:カラー画像 -1:カラー＋アルファ
img = cv2.imread('Images/mesh.png',1)
height,width,_=img.shape

#移動前の点
pts1 = np.float32([[30,30],[240,30],[60,240]])
#上記の点の移動先
pts2 = np.float32([[30,30],[240,80],[60,240]])

##移動前後の3点の位置に円を描画
img2=img.copy();
for i in range(3):
    img2 = cv2.circle(img2,(int(pts1[i,0]),int(pts1[i,1])), 10, (0,0,255),-1)
    img = cv2.circle(img,(int(pts1[i,0]),int(pts1[i,1])), 10, (0,0,255),-1)
for i in range(3):
    img2 = cv2.circle(img2,(int(pts2[i,0]),int(pts2[i,1])), 5, (255,0,0),-1)
    
#変形の変換行列
M=cv2.getAffineTransform(pts1,pts2)
#変形
dst=cv2.warpAffine(img,M,(width,height))

#画像をWindowで表示
cv2.imshow('image',img2)
cv2.imshow('result',dst)
#キー入力を待つ
cv2.waitKey(0)
#何かしらのキーを打つとウィンドウを閉じる
cv2.destroyAllWindows()
#全ての処理を終了する前にちょっと待つ。
cv2.waitKey(1)

### 補足
3点のうち1点を移動した結果(赤点→青点)、各辺の平行性を保ったまま変形が行われる。<br>
<img src="./Results/affine_result.png" width="400" >
アフィン変換というのは変換前後で平行性を保たれる変換方法のこと。これまで扱ってきたスケーリングや並進、回転も変換前後の平行性は保たれているのでアフィン変換の一種。これの他にも最も右の図のように平行四辺形状の変形もアフィン変換に含まれる。<br>
<img src="./Results/affine.png" width="700" >
アフィン変換用の行列を作るには移動前後の3点をあらかじめ決めておき、getAffineTransformに渡すだけでOK。
<br>
このサンプルでは1点だけを動かしているが、もちろん3点とも動かして変形することも可能。
<br>
アイソメトリックなイラストを作成するのに便利。<br>
【参考サイト】<br>
https://www.sidethree.co.jp/blog/memo/201904.html





## 射影変換(変形)

In [ ]:
import cv2
import numpy as np
#画像を読み込む。第二引数は 0:グレースケール、1:カラー画像 -1:カラー＋アルファ
img = cv2.imread('Images/mesh.png',1)
height,width,_=img.shape

#移動前の点
pts1 = np.float32([[30,30],[240,30],[240,240],[60,240]])
#上記の点の移動先
pts2 = np.float32([[70,60],[190,80],[230,220],[70,210]])

#移動前後の4点の位置に円を描画
img2=img.copy();
for i in range(4):
    img2 = cv2.circle(img2,(int(pts1[i,0]),int(pts1[i,1])), 10, (0,0,255),-1)
    img = cv2.circle(img,(int(pts1[i,0]),int(pts1[i,1])), 10, (0,0,255),-1)
for i in range(4):
    img2 = cv2.circle(img2,(int(pts2[i,0]),int(pts2[i,1])), 6, (255,0,0),-1)
    
#射影行列の算出
M=cv2.getPerspectiveTransform(pts1,pts2)
#射影変換
dst=cv2.warpPerspective(img,M,(width,height))

#画像をWindowで表示
cv2.imshow('image',img2)
cv2.imshow('result',dst)
#キー入力を待つ
cv2.waitKey(0)
#何かしらのキーを打つとウィンドウを閉じる
cv2.destroyAllWindows()
#全ての処理を終了する前にちょっと待つ。
cv2.waitKey(1)

### 補足
下図は4点を移動させ平行性を保たない変形を行なった結果。<br>
アフィン変換と異なり3D空間の立体感や奥行き感を出すことも可能。<br>
<img src="./Results/projective.png" width="400" >
<br>
画像の読み込みから変化までの手順はアフィン変換とおよそ同じだが、変換行列の作成には<b>getPerspectiveTransform</b>、変換には<b>warpPerspective</b>を用いる点は異なる。

## 射影変換(斜め→正面)

In [ ]:
import cv2
import numpy as np
#画像を読み込む。第二引数は 0:グレースケール、1:カラー画像 -1:カラー＋アルファ
img = cv2.imread('Images/marker.jpg',1)

#マーカー領域を正面に変換した後の画像サイズを適当に決める。
height=240
width=240

#上記のコードとの違いは点の座標と扱う画像だけ！
#移動前の点
pts1 = np.float32([[64,68],[182,58],[227,168],[78,193]])
#上記の点の移動先
pts2 = np.float32([[0,0],[width,0],[width,height],[0,height]])

#移動前後の4点の位置に円を描画
img2=img.copy();
for i in range(4):
    img2 = cv2.circle(img2,(int(pts1[i,0]),int(pts1[i,1])), 10, (0,0,255),-1)
for i in range(4):
    img2 = cv2.circle(img2,(int(pts2[i,0]),int(pts2[i,1])), 10, (255,0,0),-1)
    
#射影行列の算出
M=cv2.getPerspectiveTransform(pts1,pts2)
#射影変換
dst=cv2.warpPerspective(img,M,(width,height))

#マーカの黒枠の太さ(マーカー全体の1/4)
marginH=height//4;
marginW=width//4;

#黒枠の内側だけをクリッピング
dst2=dst[marginH:marginH*3,marginW:marginW*3]

#見やすいように240x240に変換
dst2=cv2.resize(dst2,(240,240),interpolation = cv2.INTER_CUBIC)

#画像をWindowで表示
cv2.imshow('image',img2)
cv2.imshow('result',dst)
cv2.imshow('pattern',dst2)
#キー入力を待つ
cv2.waitKey(0)
#何かしらのキーを打つとウィンドウを閉じる
cv2.destroyAllWindows()
#全ての処理を終了する前にちょっと待つ。
cv2.waitKey(1)

### 補足
斜めに配置されたオブジェクトを正面からの見た目に変換することも可能。<br>
<img src="./Results/hiro_projective.png" width="600" >
<br>
ARライブラリとしてお馴染みのARToolKitでは黒枠の四隅の検出を行なった後、正面の画像に変換を行い、さらに内側のパターンを抜き出してパターンマッチングによるマーカーの認識を行なっている。<br>
この技術はAR以外にもポストイット(付箋)や名刺を読み取り画像として保存するアプリなどでも活用されている。<br>
【参考サイト】<br>
https://blog.empathywriting.com/seminar/post-it-plus.html

# おまけ：アフィン変換の移動後の点をマウス操作で指定

In [ ]:
import cv2
import numpy as np

index=0

# マウス操作に関する処理
def mousePos(event,x,y,flags,param):
    global index
    #index番目(0~3)
    if index<3:
        #マウスの位置をindex番目の点の表にする
        pts2[index,0]=x
        pts2[index,1]=y
    #マウスをクリックしたら適当なindexにする
    if event == cv2.EVENT_LBUTTONUP:
        index=100

#画像を読み込む。第二引数は 0:グレースケール、1:カラー画像 -1:カラー＋アルファ
img = cv2.imread('Images/mesh.png',1)
height,width,_=img.shape

#移動前後の点(マウスで動かすから適当でOK)
pts1 = np.float32([[0,0],[width,0],[0,height]])
pts2 = np.float32([[0,0],[width,10],[0,height]])

#表示する前にwindow名をつける
cv2.namedWindow('image')
#上記windowにマウスアクションを対応させる
cv2.setMouseCallback('image',mousePos)
 
while (True):
    #指定した点を表示
    img2=img.copy();
    for i in range(3):
        img2 = cv2.circle(img2,(int(pts1[i,0]),int(pts1[i,1])), 10, (0,0,255),-1)
    for i in range(3):
        img2 = cv2.circle(img2,(int(pts2[i,0]),int(pts2[i,1])), 4, (255,0,0),-1)
    #アフィン変換
    M=cv2.getAffineTransform(pts1,pts2)
    dst=cv2.warpAffine(img,M,(width,height))
    
    #画像をWindowで表示
    cv2.imshow('image',img2)
    cv2.imshow('result',dst)
    #30ミリ秒、キー入力を待つ
    key = cv2.waitKey(30) & 0xFF
    #qを入力した場合は終了
    #0~2を押すと対応する点がマウスに追従するようになる。
    #画面をクリックしたらマウスに追従しなくなる。(mousePos内参照)
    if key ==ord('q') :
        break
    elif key == ord('0'):
        index=0
    elif key == ord('1'):
        index=1
    elif key==ord('2'):
        index=2
    
        
#キャプチャを終了
cv2.waitKey(1)
#全てのウィンドウを閉じる
cv2.destroyAllWindows()
#フリーズ対策として終わる前にちょっと待つ
cv2.waitKey(1)

### 補足
imageウィンドウを選択し0~2のキーを押した後にマウスを移動すると対応する点が移動してresultウィンドウ内の画像が変形する。<br>
[0] : 左上の点の移動<br>
[1] : 右上の点の移動<br>
[2]  :左下の点の移動<br>
マウスをクリックすると移動は終了。<br>
[q]キーをクリックするとウィンドウを閉じることができる